# HEC-RAS 2D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the HdfBase, HdfUtils, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, and HdfResultsXsec classes to streamline data extraction, processing, and visualization.


In [1]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas', 'pyproj', 'shapely', 'xarray', 'rasterio']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import logging
from pathlib import Path
import rasterio
from rasterio.plot import show


In [2]:
# Install ras-commander if you are not in a dev environment. 
# install_module(ras-commander)

## Importing ras-commander flexibly (from package or local dev copy)

In [ ]:
# Flexible Import for RAS Commander
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger
except ImportError:
    # If the import fails, add the parent directory to the Python path
    print("Using Local Dev Copy")
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger

print("ras_commander imported successfully")

In [ ]:
# Download the BaldEagleCrkMulti2D project from HEC and run plan 01

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
bald_eagle_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = bald_eagle_path / "BaldEagleDamBrk.p06.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    ras_examples = RasExamples()
    ras_examples.extract_project(["BaldEagleCrkMulti2D"])

    # Initialize custom Ras object
    bald_eagle = RasPrj()

    # Initialize the RAS project using the custom ras object
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    logging.info(f"Bald Eagle project initialized with folder: {bald_eagle.project_folder}")
    
    logging.info(f"Bald Eagle object id: {id(bald_eagle)}")
    
    # Define the plan number to execute
    plan_number = "06"

    # Update run flags for the project
    RasPlan.update_run_flags(
        plan_number,
        geometry_preprocessor=True,
        unsteady_flow_simulation=True,
        run_sediment=False,
        post_processor=True,
        floodplain_mapping=False,
        ras_object=bald_eagle
    )

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number, ras_object=bald_eagle)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("BaldEagleCrkMulti2D.p06.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    bald_eagle = RasPrj()
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    plan_number = "06"

In [ ]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for bald_eagle project
print("Plan DataFrame for bald_eagle project:")
display(bald_eagle.plan_df)

# Display geom_df for bald_eagle project
print("\nGeometry DataFrame for bald_eagle project:")
display(bald_eagle.geom_df)

# Get the plan HDF path
plan_number = "06"  # Assuming we're using plan 01 as in the previous code
plan_hdf_path = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

# Get the geometry file number from the plan DataFrame
geom_file = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'Geom File'].values[0]
geom_number = geom_file[1:]  # Remove the 'g' prefix

# Get the geometry HDF path
geom_hdf_path = bald_eagle.geom_df.loc[bald_eagle.geom_df['geom_number'] == geom_number, 'hdf_path'].values[0]

print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")

In [6]:
# Define the HDF input path as Plan Number

plan_number = "06"  # Assuming we're using plan 01 as in the previous code


RasHdfUtils
| Method Name | Description |
|-------------|-------------|
| get_attrs | Converts attributes from a HEC-RAS HDF file into a Python dictionary for a given attribute path |
| get_root_attrs | Returns attributes at root level of HEC-RAS HDF file |
| get_hdf_paths_with_properties | Gets all paths in the HDF file with their properties |
| get_group_attributes_as_df | Gets attributes of a group in the HDF file as a DataFrame |
| get_hdf_filename | Gets the HDF filename from various input types |
| get_runtime_data | Extracts runtime and compute time data from a single HDF file |


In [ ]:
# Get HDF Paths with Properties (For Exploring HDF Files)
HdfBase.get_dataset_info(plan_number, ras_object=bald_eagle, group_path="/")

In [ ]:
# Example: Extract runtime and compute time data
print("\nExample 2: Extracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_input=plan_number, ras_object=bald_eagle)
if runtime_df is not None:
    display(runtime_df)
else:
    print("No runtime data found.")

runtime_df example output: 

| Plan Name                        | File Name                     | Simulation Start Time | Simulation End Time | Simulation Duration (s) | Simulation Time (hr) | Completing Geometry (hr) | Preprocessing Geometry (hr) | Completing Event Conditions (hr) | Unsteady Flow Computations (hr) | Complete Process (hr) | Unsteady Flow Speed (hr/hr) | Complete Process Speed (hr/hr) |
|----------------------------------|-------------------------------|-----------------------|---------------------|-------------------------|-----------------------|--------------------------|------------------------------|----------------------------------|----------------------------------|-----------------------|------------------------------|----------------------------------|
| Gridded Precip - Infiltration    | BaldEagleDamBrk.p06.hdf      | 09Sep2018 00:00:00    | 14Sep2018 00:00:00  | 432000.0                | 120.0                 | N/A                      | 0.000113                     | N/A                              | 0.074436                        | 0.080951              | 1612.126776                  | 1482.386368                      |

Table of all the functions in the RasGeomHdf class from the ras_commander/RasGeomHdf.py file:

| Function Name | Description |
|---------------|-------------|
| projection | Returns the projection of the RAS geometry as a pyproj.CRS object |
| get_geom_attrs | Returns base geometry attributes from a HEC-RAS HDF file |

| mesh_area_names | Returns a list of the 2D mesh area names of the RAS geometry |
| get_geom_2d_flow_area_attrs | Returns geometry 2d flow area attributes from a HEC-RAS HDF file |
| mesh_areas | Returns 2D flow area perimeter polygons |
| mesh_cell_polygons | Returns 2D flow mesh cell polygons |
| mesh_cell_points | Returns 2D flow mesh cell points |
| mesh_cell_faces | Returns 2D flow mesh cell faces |

| get_geom_structures_attrs | Returns geometry structures attributes from a HEC-RAS HDF file |




| bc_lines | Returns 2D mesh area boundary condition lines |
| breaklines | Returns 2D mesh area breaklines |



| refinement_regions | Returns 2D mesh area refinement regions |
| structures | Returns the model structures |
| reference_lines_names | Returns reference line names |
| reference_points_names | Returns reference point names |
| reference_lines | Returns the reference lines geometry and attributes |
| reference_points | Returns the reference points geometry and attributes |
| cross_sections | Returns the model 1D cross sections |
| river_reaches | Returns the model 1D river reach lines |
| cross_sections_elevations | Returns the model cross section elevation information |

In [ ]:
# For all of the RasGeomHdf Class Functions, we will use geom_hdf_path
print(geom_hdf_path)

# For the example project, plan 06 is associated with geometry 09
# If you want to call the geometry by number, call RasHdfGeom functions with a number
# Otherwise, if you want to look up geometry hdf path by plan number, follow the logic in the previous code cells

In [ ]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Projection: {projection}")
else:
    print("No projection information found.")

In [ ]:
# Use HdfPlan for geometry-related operations
print("\nExample: Extracting Geometry Information")
geom_attrs = HdfPlan.get_geometry_information(geom_hdf_path)
display(geom_attrs)


geom_attrs output: 

| Complete Geometry | Extents | Geometry Time | Infiltration Date Last Modified | Infiltration File Date | Infiltration Filename | Infiltration Layername | Land Cover Date Last Modified | Land Cover File Date | Land Cover Filename | ... | Percent Impervious Date Last Modified | Percent Impervious File Date | Percent Impervious Filename | Percent Impervious Layername | SI Units | Terrain File Date | Terrain Filename | Terrain Layername | Title | Version |
|-------------------|---------|---------------|---------------------------------|-----------------------|----------------------|------------------------|------------------------------|----------------------|---------------------|-----|--------------------------------------|-----------------------------|----------------------------|------------------------------|----------|-------------------|------------------|-------------------|-------|---------|
| 0                 | True    | [1960041.35636708, 2092643.59732271, 285497.89...] | 27Oct2024 20:09:19 | 11MAR2022 13:52:44 | 24NOV2020 13:24:58 | .\Soils Data\Infiltration.hdf | Infiltration | 11MAR2022 13:45:08 | 11MAR2022 13:45:08 | .\Land Classification\LandCover.hdf | ... | 11MAR2022 13:45:08 | 11MAR2022 13:45:08 | .\Land Classification\LandCover.hdf | LandCover | False | 09FEB2015 08:26:58 | .\Terrain\Terrain50.hdf | Terrain50 | Single 2D Area - Internal Dam Structure | 1.0.20 (20Sep2024) |

In [ ]:
# Use HdfMesh for geometry-related operations
print("\nExample 3: Listing 2D Flow Area Names")
flow_area_names = HdfMesh.get_mesh_area_names(geom_hdf_path)
print("2D Flow Area Name (returned as list):\n", flow_area_names)
# Note: this is returned as a list because it is used internally by other functions.  

In [ ]:
# Example: Get 2D Flow Area Attributes (get_mesh_area_attributes)
print("\nExample: Extracting 2D Flow Area Attributes")
flow_area_attributes = HdfMesh.get_mesh_area_attributes(geom_hdf_path)
display(flow_area_attributes)

flow_area_df:

Value
| Name                        | b'BaldEagleCr' |
|-----------------------------|-----------------|
| Locked                      | 0               |
| Mann                        | 0.04            |
| Multiple Face Mann n       | 0               |
| Composite LC               | 0               |
| Cell Vol Tol               | 0.01            |
| Cell Min Area Fraction      | 0.01            |
| Face Profile Tol           | 0.01            |
| Face Area Tol              | 0.01            |
| Face Conv Ratio            | 0.02            |
| Laminar Depth              | 0.2             |
| Min Face Length Ratio      | 0.05            |
| Spacing dx                 | 250.0           |
| Spacing dy                 | 250.0           |
| Shift dx                   | NaN             |
| Shift dy                   | NaN             |
| Cell Count                 | 18066           |

In [ ]:
# Example: Get 2D Flow Area Perimeter Polygons (get_mesh_areas)
print("\nExample: Extracting 2D Flow Area Perimeter Polygons")
mesh_areas = HdfMesh.get_mesh_areas(geom_hdf_path, ras_object=bald_eagle)

# Plot the 2D Flow Area Perimeter Polygons
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))
mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none')

# Add labels for each polygon
for idx, row in mesh_areas.iterrows():
    centroid = row.geometry.centroid
    # Check if 'Name' column exists, otherwise use a default label
    label = row.get('Name', f'Area {idx}')
    ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

plt.title('2D Flow Area Perimeter Polygons')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.tight_layout()
plt.show()

In [ ]:
# Example: Extract mesh cell faces
print("\nExample: Extracting mesh cell faces")

# Get mesh cell faces
mesh_cell_faces = HdfMesh.get_mesh_cell_faces(geom_hdf_path, ras_object=bald_eagle)

# Display the first few rows of the mesh cell faces DataFrame
print("First few rows of mesh cell faces:")
display(mesh_cell_faces.head())

mesh_cell_faces geodataframe:

flow_area_df:

| mesh_name    | face_id | geometry                                           |
|--------------|---------|----------------------------------------------------|
| BaldEagleCr  | 0       | LINESTRING (2042125 351625, 2042375 351625)      |
| BaldEagleCr  | 1       | LINESTRING (2042375 351625, 2042375 351875)      |
| BaldEagleCr  | 2       | LINESTRING (2042375 351875, 2042125 351875)      |
| BaldEagleCr  | 3       | LINESTRING (2042125 351875, 2042125 351625)      |
| BaldEagleCr  | 4       | LINESTRING (2042375 351375, 2042375 351625)      |

In [ ]:
from matplotlib.collections import LineCollection
import numpy as np

# Plot the mesh cell faces more efficiently
fig, ax = plt.subplots(figsize=(12, 8))

# Convert all geometries to numpy arrays at once for faster plotting
lines = [list(zip(*line.xy)) for line in mesh_cell_faces.geometry]
lines_collection = LineCollection(lines, colors='blue', linewidth=0.5, alpha=0.5)
ax.add_collection(lines_collection)

# Set plot title and labels
plt.title('Mesh Cell Faces')
plt.xlabel('Easting')
plt.ylabel('Northing')

# Calculate centroids once and store as numpy arrays
centroids = np.array([[geom.centroid.x, geom.centroid.y] for geom in mesh_cell_faces.geometry])

# Create scatter plot with numpy arrays
scatter = ax.scatter(
    centroids[:, 0],
    centroids[:, 1], 
    c=mesh_cell_faces['face_id'],
    cmap='viridis',
    s=1,
    alpha=0.5
)
plt.colorbar(scatter, label='Face ID')

# Set axis limits based on data bounds
ax.set_xlim(centroids[:, 0].min(), centroids[:, 0].max())
ax.set_ylim(centroids[:, 1].min(), centroids[:, 1].max())

plt.tight_layout()
plt.show()

# Calculate and display some statistics
print("\nMesh Cell Faces Statistics:")
print(f"Total number of cell faces: {len(mesh_cell_faces)}")
print(f"Number of unique meshes: {mesh_cell_faces['mesh_name'].nunique()}")


In [ ]:
# Function to find the nearest cell face to a given point
def find_nearest_cell_face(point, cell_faces_df):
    """
    Find the nearest cell face to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_faces_df (GeoDataFrame): DataFrame containing cell face linestrings.

    Returns:
        int: The face_id of the nearest cell face.
        float: The distance to the nearest cell face.
    """
    # Calculate distances from the input point to all cell faces
    distances = cell_faces_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the face_id and distance of the nearest cell face
    nearest_face_id = cell_faces_df.loc[nearest_index, 'face_id']
    nearest_distance = distances[nearest_index]

    return nearest_face_id, nearest_distance

# Example usage
print("\nExample: Finding the nearest cell face to a given point")

# Create a sample point (you can replace this with any point of interest)
from shapely.geometry import Point
from geopandas import GeoDataFrame

# Get the projection from the geometry file
# projection = HdfUtils.get_projection(hdf_path=geom_hdf_path) # This was done in a previous code cell
if projection:
    print(f"Using projection: {projection}")
else:
    print("No projection information found. Using default CRS.")
    projection = "EPSG:4326"  # Default to WGS84 if no projection is found

# Create the sample point with the correct CRS
sample_point = GeoDataFrame({'geometry': [Point(2042250, 351750)]}, crs=projection)

if not mesh_cell_faces.empty and not sample_point.empty:
    # Ensure the CRS of the sample point matches the mesh_cell_faces
    if sample_point.crs != mesh_cell_faces.crs:
        sample_point = sample_point.to_crs(mesh_cell_faces.crs)
    
    nearest_face_id, distance = find_nearest_cell_face(sample_point.geometry.iloc[0], mesh_cell_faces)
    print(f"Nearest cell face to point {sample_point.geometry.iloc[0].coords[0]}:")
    print(f"Face ID: {nearest_face_id}")
    print(f"Distance: {distance:.2f} units")

    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all cell faces
    mesh_cell_faces.plot(ax=ax, color='blue', linewidth=0.5, alpha=0.5, label='Cell Faces')
    
    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')
    
    # Plot the nearest cell face
    nearest_face = mesh_cell_faces[mesh_cell_faces['face_id'] == nearest_face_id]
    nearest_face.plot(ax=ax, color='green', linewidth=2, alpha=0.7, label='Nearest Face')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell Face to Sample Point')
    
    # Add legend and grid
    ax.legend()
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("Unable to perform nearest cell face search due to missing data.")



In [ ]:
# Example: Extract Cell Polygons
print("\nExample 6: Extracting Cell Polygons")
cell_polygons_df = HdfMesh.get_mesh_cell_polygons(geom_hdf_path, ras_object=bald_eagle)

cell_polygons_df:

| mesh_name    | cell_id | geometry                                      |
|--------------|---------|-----------------------------------------------|
| BaldEagleCr  | 0       | POLYGON ((2082875 370625, 2082723.922 370776.0... |
| BaldEagleCr  | 1       | POLYGON ((2083125 370625, 2083125 370844.185, ... |
| BaldEagleCr  | 2       | POLYGON ((2083375 370625, 2083375 370886.638, ... |
| BaldEagleCr  | 3       | POLYGON ((2083625 370625, 2083625 370925.693, ... |
| BaldEagleCr  | 4       | POLYGON ((2083875 370625, 2083875 370958.588, ... |

In [ ]:
# Plot Cell Polygons
if not cell_polygons_df.empty:
    display(cell_polygons_df.head())
else:
    print("No Cell Polygons found.")

# Plot cell polygons
if not cell_polygons_df.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot cell polygons
    cell_polygons_df.plot(ax=ax, edgecolor='blue', facecolor='none')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Polygons')
    
    # Add grid
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No cell polygon data available for plotting.")

In [ ]:
# Extract Cell Info
print("\nExample 5: Extracting Cell Info")
cell_info_df = HdfMesh.get_mesh_cell_points(geom_hdf_path, ras_object=bald_eagle)

cell_info_df: 

| mesh_name    | cell_id | geometry                          |
|--------------|---------|-----------------------------------|
| BaldEagleCr  | 0       | POINT (2083000 370750)           |
| BaldEagleCr  | 1       | POINT (2083250 370750)           |
| BaldEagleCr  | 2       | POINT (2083500 370750)           |
| BaldEagleCr  | 3       | POINT (2083750 370750)           |
| BaldEagleCr  | 4       | POINT (2084000 370750)           |

In [ ]:
# Plot Cell Info
if not cell_info_df.empty:
    display(cell_info_df.head())
else:
    print("No Cell Info found.")

# Plot cell centers
import matplotlib.pyplot as plt

if not cell_info_df.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot cell centers
    cell_info_df.plot(ax=ax, color='red', markersize=5)
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Centers')
    
    # Add grid
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No cell data available for plotting.")


In [ ]:
# Function to find the nearest cell center to a given point
def find_nearest_cell(point, cell_centers_df):
    """
    Find the nearest cell center to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_centers_df (GeoDataFrame): DataFrame containing cell center points.

    Returns:
        int: The cell_id of the nearest cell.
        float: The distance to the nearest cell center.
    """
    # Calculate distances from the input point to all cell centers
    distances = cell_centers_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the cell_id and distance of the nearest cell
    nearest_cell_id = cell_centers_df.loc[nearest_index, 'cell_id']
    nearest_distance = distances[nearest_index]

    return nearest_cell_id, nearest_distance

# Example usage
print("\nExample: Finding the nearest cell to a given point")

# Create a sample point (you can replace this with any point of interest)
from shapely.geometry import Point
from geopandas import GeoDataFrame

# Get the projection from the geometry file
# projection = HdfUtils.get_projection(hdf_path=geom_hdf_path) # This was done in a previous code cell
if projection:
    print(f"Using projection: {projection}")
else:
    print("No projection information found. Using default CRS.")
    projection = "EPSG:4326"  # Default to WGS84 if no projection is found

# Create the sample point with the correct CRS
sample_point = GeoDataFrame({'geometry': [Point(2083500, 370800)]}, crs=projection)

if not cell_info_df.empty and not sample_point.empty:
    # Ensure the CRS of the sample point matches the cell_info_df
    if sample_point.crs != cell_info_df.crs:
        sample_point = sample_point.to_crs(cell_info_df.crs)
    
    nearest_cell_id, distance = find_nearest_cell(sample_point.geometry.iloc[0], cell_info_df)
    print(f"Nearest cell to point {sample_point.geometry.iloc[0].coords[0]}:")
    print(f"Cell ID: {nearest_cell_id}")
    print(f"Distance: {distance:.2f} units")

    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all cell centers
    cell_info_df.plot(ax=ax, color='blue', markersize=5, alpha=0.5, label='Cell Centers')
    
    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')
    
    # Plot the nearest cell center
    nearest_cell = cell_info_df[cell_info_df['cell_id'] == nearest_cell_id]
    nearest_cell.plot(ax=ax, color='green', markersize=100, alpha=0.7, label='Nearest Cell')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell to Sample Point')
    
    # Add legend and grid
    ax.legend()
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("Unable to perform nearest cell search due to missing data.")


In [ ]:
# Get geometry structures attributes
print("\nGetting geometry structures attributes")
geom_structures_attrs = HdfStruc.get_geom_structures_attrs(geom_hdf_path, ras_object=bald_eagle)
if geom_structures_attrs:
    print("Geometry structures attributes:")
    for key, value in geom_structures_attrs.items():
        print(f"{key}: {value}")
else:
    print("No geometry structures attributes found.")

In [24]:
# TODO: Paths and Functions for each type of structure: 

# Getting geometry structures attributes
# Geometry structures attributes:
# Bridge/Culvert Count: 0
# Connection Count: 4
# Has Bridge Opening (2D): 0
# Inline Structure Count: 0
# Lateral Structure Count: 0

In [ ]:
# Example: Extract Boundary Condition Lines and Plot with 2D Flow Area Perimeter Polygons
print("\nExample 7: Extracting Boundary Condition Lines and Plotting with 2D Flow Area Perimeter Polygons")
bc_lines_df = HdfBndry.get_bc_lines(geom_hdf_path, ras_object=bald_eagle)

if not bc_lines_df.empty:
    display(bc_lines_df.head())
else:
    print("No Boundary Condition Lines found.")

| bc_line_id |         name         |    mesh_name    |    type    |                                           geometry                                            |
|-------------|----------------------|------------------|------------|------------------------------------------------------------------------------------------------|
|      0      |     DSNormalDepth    |   BaldEagleCr    |  External  | LINESTRING (2082004.235 364024.82, 2083193.546...)                                          |
|      1      |       DS2NormalD     |   BaldEagleCr    |  External  | LINESTRING (2084425.804 365392.892, 2084354.64...)                                          |
|      2      |   Upstream Inflow    |   BaldEagleCr    |  External  | LINESTRING (1967473.737 290973.629, 1969582.89...)                                          |

In [ ]:
# Plot Boundary Condition Lines with Perimeter
# Plot if data exists
if not bc_lines_df.empty or not mesh_areas.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')
    
    # Plot boundary condition lines
    if not bc_lines_df.empty:
        bc_lines_df.plot(ax=ax, color='red', linewidth=2, label='Boundary Condition Lines')
    
    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Boundary Condition Lines')
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No data available for plotting.")

In [ ]:
# Example: Extract Breaklines and Plot with 2D Flow Area Perimeter Polygons
print("\nExample 8: Extracting Breaklines and Plotting with 2D Flow Area Perimeter Polygons")
breaklines_gdf = HdfBndry.get_breaklines(geom_hdf_path, ras_object=bald_eagle)
if not breaklines_gdf.empty:
    display(breaklines_gdf.head())
else:
    print("No Breaklines found.")

breaklines_gdf:


| bl_id | Name      | geometry |
|-------|-----------|----------|
| 0     | SayersDam | LINESTRING (2002361.246 323707.927, 2002741.35...) |
| 1     | Lower     | LINESTRING (2060356.422 351786.819, 2060316.47...) |
| 2     | Middle    | LINESTRING (2052757.788 348470.547, 2052785.84...) |
| 3     | Upper     | LINESTRING (2045597.199 348412.994, 2045638.91...) |

In [ ]:
# Plot breaklines and 2D Flow Area Perimeter Polygons if they exist
if not breaklines_gdf.empty or not mesh_areas.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')
    
    # Plot breaklines
    if not breaklines_gdf.empty:
        breaklines_gdf.plot(ax=ax, color='blue', linewidth=2, label='Breaklines')
    
    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Breaklines')
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No data available for plotting.")

In [29]:
# INSTEAD OF hdf_input, USE plan_hdf_path or geom_hdf_path as appropriate 

In [ ]:
# Example: Get structures
structures_gdf = HdfStruc.get_structures(geom_hdf_path, ras_object=bald_eagle)
print("Structures:")
if not structures_gdf.empty:
    display(structures_gdf.head())
else:
    print("No structures found in the geometry file.")

structures_gdf: 

| Type | Mode | River | Reach | RS | Connection | Groupname | US Type | US River | US Reach | ... | US XS Mann (Count) | US BR Mann (Index) | US BR Mann (Count) | DS XS Mann (Index) | DS XS Mann (Count) | DS BR Mann (Index) | DS BR Mann (Count) | RC (Index) | RC (Count) | Profile_Data |
|------|------|-------|-------|-------|------------|-----------|----------|-----------|-----------|-----|-------------------|-------------------|-------------------|-------------------|-------------------|-------------------|-------------------|------------|------------|--------------|
| Connection | Weir/Gate/Culverts | | | | Sayers Dam | BaldEagleCr, Sayers Dam | 2D | | | ... | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | [{'Station': 0.0, 'Elevation': 683.0}, {'Stati... |

In [ ]:
# Example: Get boundary condition lines
ref_lines_gdf = HdfBndry.get_bc_lines(geom_hdf_path)
print("\nBoundary Condition Lines:")
if not ref_lines_gdf.empty:
    display(ref_lines_gdf.head())
else:
    print("No boundary condition lines found in the geometry file.")

In [ ]:
# Example: Get reference points
ref_points_gdf = HdfBndry.get_reference_points(geom_hdf_path)
print("\nReference Points:")
if not ref_points_gdf.empty:
    display(ref_points_gdf.head())
else:
    print("No reference points found in the geometry file.")

# Extract Breakline as Reference Line

We can't use a profile line, because the mesh orientation may be quite different than the direction of flow.  

Instead, use a breakline - the one named "SayersDam" should work

We can find the information specific to faces: 






# Extract Composite Results for 2D at Profile Lines to simulate Reference Lines



In [ ]:
# Example: Extract Refinement Regions
print("\nExample: Extracting Refinement Regions")

# Make sure to pass the bald_eagle object as the ras_object parameter
refinement_regions_df = HdfBndry.get_refinement_regions(geom_hdf_path, ras_object=bald_eagle)

if not refinement_regions_df.empty:
    print("Refinement Regions DataFrame:")
    display(refinement_regions_df.head())
    
    # Plot refinement regions
    fig, ax = plt.subplots(figsize=(12, 8))
    refinement_regions_df.plot(ax=ax, column='CellSize', legend=True, 
                               legend_kwds={'label': 'Cell Size', 'orientation': 'horizontal'},
                               cmap='viridis')
    ax.set_title('2D Mesh Area Refinement Regions')
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    plt.tight_layout()
    plt.show()
else:
    print("No refinement regions found in the geometry file.")

# Example: Analyze Refinement Regions
if not refinement_regions_df.empty:
    print("\nRefinement Regions Analysis:")
    print(f"Total number of refinement regions: {len(refinement_regions_df)}")
    print("\nCell Size Statistics:")
    print(refinement_regions_df['CellSize'].describe())
    
    # Group by Shape Type
    shape_type_counts = refinement_regions_df['ShapeType'].value_counts()
    print("\nRefinement Region Shape Types:")
    print(shape_type_counts)
    
    # Plot Shape Type distribution
    plt.figure(figsize=(10, 6))
    shape_type_counts.plot(kind='bar')
    plt.title('Distribution of Refinement Region Shape Types')
    plt.xlabel('Shape Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Extract Compute Messages as String
print("Extracting Compute Messages")

import h5py
import numpy as np

def extract_string_from_hdf(results_hdf_filename: str, hdf_path: str) -> str:
    """
    Extract string from HDF object at a given path

    Parameters
    ----------
    results_hdf_filename : str
        Name of the HDF file
    hdf_path : str
        Path of the object in the HDF file

    Returns
    -------
    str
        Extracted string from the specified HDF object
    """
    with h5py.File(results_hdf_filename, 'r') as hdf_file:
        try:
            hdf_object = hdf_file[hdf_path]
            if isinstance(hdf_object, h5py.Group):
                return f"Group: {hdf_path}\nContents: {list(hdf_object.keys())}"
            elif isinstance(hdf_object, h5py.Dataset):
                data = hdf_object[()]
                if isinstance(data, bytes):
                    return data.decode('utf-8')
                elif isinstance(data, np.ndarray) and data.dtype.kind == 'S':
                    return [v.decode('utf-8') for v in data]
                else:
                    return str(data)
            else:
                return f"Unsupported object type: {type(hdf_object)}"
        except KeyError:
            return f"Path not found: {hdf_path}"

try:
    results_summary_string = extract_string_from_hdf(plan_hdf_path, '/Results/Summary/Compute Messages (text)')
    print("Compute Messages:")
    
    # Parse and print the compute messages in a more visually friendly way
    messages = results_summary_string[0].split('\r\n')
    
    for message in messages:
        if message.strip():  # Skip empty lines
            if ':' in message:
                key, value = message.split(':', 1)
                print(f"{key.strip():40} : {value.strip()}")
            else:
                print(f"\n{message.strip()}")
    
    # Print computation summary in a table format
    print("\nComputation Summary:")
    print("-" * 50)
    print(f"{'Computation Task':<30} {'Time':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Task' in line:
            task, time = line.split('\t')
            print(f"{task:<30} {time:<20}")
    
    print("\nComputation Speed:")
    print("-" * 50)
    print(f"{'Task':<30} {'Simulation/Runtime':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Speed' in line:
            task, speed = line.split('\t')
            print(f"{task:<30} {speed:<20}")

except Exception as e:
    print(f"Error extracting compute messages: {str(e)}")
    print("\nNote: If 'Results/Summary Output' is not in the file structure, it might indicate that the simulation didn't complete successfully or the results weren't saved properly.")

 



In [ ]:
# Advanced Compute Messages Example - TODO: Move this function into a class of the library 
import pandas as pd
import re
import matplotlib.pyplot as plt
import geopandas as gpd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

def parse_2d_compute_messages(compute_messages):
    """
    Parse 2D compute messages to extract data lines, clean the data, 
    and retrieve top 20 cells with the highest error.

    Parameters:
        compute_messages (list or str): The raw compute messages.

    Returns:
        tuple: A tuple containing the parsed compute messages string and the main DataFrame.
    """
    try:
        # Handle both list and string inputs
        if isinstance(compute_messages, list):
            compute_messages = '\n'.join(compute_messages)
        elif not isinstance(compute_messages, str):
            logging.error(f"Unexpected type for compute_messages: {type(compute_messages)}")
            return "", pd.DataFrame()

        # Split the message into lines
        lines = compute_messages.split('\n')
        logging.info("Successfully split compute messages into lines.")
        
        # Initialize lists to store parsed data
        data_lines = []
        header_lines = []
        footer_lines = []
        
        # Regular expression to match timestamp lines
        timestamp_pattern = re.compile(r'^\d{2}[A-Z]{3}\d{4}\s+\d{2}:\d{2}:\d{2}')
        logging.debug("Compiled timestamp regular expression.")
        
        data_started = False
        for line in lines:
            stripped_line = line.strip()
            if timestamp_pattern.match(stripped_line):
                data_started = True
                # Split the line and add to data_lines
                parts = stripped_line.split()
                if len(parts) >= 8:  # Ensure we have all expected columns
                    # Combine Date and Time into 'Date and Time'
                    date_time = f"{parts[0]} {parts[1]}"
                    location = parts[2]
                    cell_type = f"{parts[3]} {parts[4]}"
                    cell_number = parts[5]
                    wsel = parts[6]
                    error = parts[7]
                    iterations = parts[8] if len(parts) > 8 else None
                    data_lines.append([date_time, location, cell_type, cell_number, wsel, error, iterations])
                    logging.debug(f"Parsed data line: {data_lines[-1]}")
                else:
                    logging.warning(f"Line skipped due to insufficient parts: {stripped_line}")
            elif not data_started:
                header_lines.append(stripped_line)
            elif data_started and not stripped_line:
                data_started = False
            elif not data_started:
                footer_lines.append(stripped_line)
        
        # Create DataFrame from data lines
        df = pd.DataFrame(
            data_lines, 
            columns=['Date and Time', 'Location', 'Cell Type', 'Cell Number', 'WSEL', 'ERROR', 'ITERATIONS']
        )
        logging.info("Created DataFrame from parsed data lines.")
        
        # Clean and convert columns to appropriate types
        df['Cell Number'] = (
            pd.to_numeric(df['Cell Number'].replace('#', pd.NA), errors='coerce')
            .fillna(-1)
            .astype('Int64')
        )
        df['WSEL'] = pd.to_numeric(df['WSEL'], errors='coerce')
        df['ERROR'] = pd.to_numeric(df['ERROR'], errors='coerce')
        df['ITERATIONS'] = pd.to_numeric(df['ITERATIONS'], errors='coerce').astype('Int64')
        logging.info("Converted DataFrame columns to appropriate types.")
        
        # Get top 20 cells with highest error
        top_20_cells = (
            df.sort_values('ERROR', ascending=False)
            .drop_duplicates('Cell Number')
            .head(20)
        )
        
        # Construct the reordered message
        reordered_message = '\n'.join(header_lines + 
                                      ['\nTop 20 Cells with Highest Error:'] + 
                                      [' '.join(map(str, row)) for row in top_20_cells.values] + 
                                      ['\n'] + footer_lines)
        
        logging.info("Reordered compute messages.")
        
        return reordered_message, df
    except Exception as e:
        logging.error(f"Error parsing compute messages: {e}")
        return "", pd.DataFrame()

# Use the function to parse compute messages
parsed_messages, df = parse_2d_compute_messages(results_summary_string)

print(parsed_messages)
print(df)

# Get top 20 cells with highest error
if not df.empty and 'ERROR' in df.columns:
    top_20_cells = (
        df.sort_values('ERROR', ascending=False)
        .drop_duplicates('Cell Number')
        .head(20)
    )
else:
    logging.warning("Unable to get top 20 cells with highest error. DataFrame is empty or 'ERROR' column is missing.")
    top_20_cells = pd.DataFrame()

# Example: Get 2D Flow Area Perimeter Polygons (mesh_areas)
print("\nExample: Extracting 2D Flow Area Perimeter Polygons")
mesh_areas = HdfMesh.get_mesh_areas(geom_hdf_path, ras_object=bald_eagle)

print("\n2D Flow Area Groups and Perimeters:")
if not mesh_areas.empty:
    print("Available columns:", mesh_areas.columns.tolist())
    
    # Display the first few rows of the mesh_areas DataFrame
    print("\nFirst few rows of mesh_areas DataFrame:")
    display(mesh_areas.head())
else:
    print("No 2D Flow Area groups found in the HDF file.")

# Use the previously extracted cell_polygons_df
print("\nTop 20 Cell Polygons:")
if 'cell_polygons_df' in locals() and not cell_polygons_df.empty and not top_20_cells.empty:
    # Get the cell numbers from top_20_cells
    top_20_cell_numbers = top_20_cells['Cell Number'].tolist()
    
    # Filter cell_polygons_df to only include top 20 cells
    top_20_cell_polygons = cell_polygons_df[cell_polygons_df['cell_id'].isin(top_20_cell_numbers)]
    
    display(top_20_cell_polygons)

    # Plot top 20 cell polygons and mesh areas
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot mesh areas
    mesh_areas.plot(ax=ax, edgecolor='red', facecolor='none', alpha=0.5, label='Mesh Areas')
    
    # Plot top 20 cell polygons
    top_20_cell_polygons.plot(ax=ax, edgecolor='blue', facecolor='none', alpha=0.7, label='Top 20 Error Cells')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Perimeters and Top 20 Cell Polygons')
    
    # Add legend
    ax.legend()
    
    # Add grid
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("No Cell Polygons found or no top 20 cells with highest error available.")
    print("Unable to plot cell polygons.")

In [ ]:
# Exploratory Example for Debugging or New Features: List all paths, groups, and attributes under "/Results/Unsteady/Summary/Volume Accounting"
HdfBase.get_dataset_info(plan_hdf_path, "/Results/Unsteady/Summary/Volume Accounting")

In [ ]:
# Example 12: Extract Plan Parameters and Volume Accounting
print("\nExample 12: Extracting Plan Parameters and Volume Accounting Data")

# Extract plan parameters
plan_parameters_df = HdfPlan.get_plan_parameters(plan_hdf_path)

# Extract volume accounting data
volume_accounting_df = HdfResultsPlan.get_volume_accounting(plan_hdf_path)

print("\nPlan Parameters DataFrame:")
display(plan_parameters_df)

print("\nVolume Accounting DataFrame:")
display(volume_accounting_df)

------

# RasPlanHdf Class Functions

-----

In [ ]:
# Example: Get plan start time
start_time = HdfPlan.get_plan_start_time(plan_hdf_path)
print(f"Simulation start time: {start_time}")

Simulation start time: 2018-09-09 00:00:00

In [ ]:
# Example: Get plan end time
end_time = HdfPlan.get_plan_end_time(plan_hdf_path)
print(f"Simulation end time: {end_time}")

Simulation end time: 2018-09-14 00:00:00

In [ ]:
# Example: Get maximum iteration count for mesh cells
max_iter_df = HdfResultsMesh.get_mesh_max_iter(plan_hdf_path)
print("\nMesh Max Iterations:")
print(max_iter_df.attrs)
display(max_iter_df.head())

max_iter_df:

| mesh_name | cell_id | cell_last_iteration | geometry |
|-----------|---------|--------------------| ---------|
| BaldEagleCr | 0 | 0 | POINT (2083000 370750) |
| BaldEagleCr | 1 | 0 | POINT (2083250 370750) |
| BaldEagleCr | 2 | 0 | POINT (2083500 370750) |
| BaldEagleCr | 3 | 2 | POINT (2083750 370750) |
| BaldEagleCr | 4 | 0 | POINT (2084000 370750) |

In [ ]:
# Get maximum iteration count for mesh cells
from ras_commander.HdfResultsMesh import HdfResultsMesh

max_iter_gdf = HdfResultsMesh.get_mesh_max_iter(plan_hdf_path)

print("max_iter_df")
print(max_iter_df)

mesh_max_iter_df:

| mesh_name | cell_id | cell_last_iteration | geometry |
|-----------|---------|--------------------| ---------|
| BaldEagleCr | 0 | 0 | POINT (2083000 370750) |
| ... | ... | ... | ... |
| BaldEagleCr | 19592 | 0 | POINT (1978423.032 300718.897) |


[19597 rows x 4 columns]

In [ ]:
# Get cell coordinates 
cell_coords = HdfMesh.get_mesh_cell_points(plan_hdf_path)
display(cell_coords)

In [ ]:
# Plot Mesh Max Iterations

# Extract x and y coordinates from the geometry column
max_iter_df['x'] = max_iter_df['geometry'].apply(lambda geom: geom.x if geom is not None else None)
max_iter_df['y'] = max_iter_df['geometry'].apply(lambda geom: geom.y if geom is not None else None)

# Remove rows with None coordinates
max_iter_df = max_iter_df.dropna(subset=['x', 'y'])

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_iter_df['x'], max_iter_df['y'], 
                     c=max_iter_df['cell_last_iteration'], 
                     cmap='viridis', 
                     s=1)

# Customize the plot
ax.set_title('Max Iterations per Cell')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Iterations')

# Show the plot
plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the dataframe:")
display(max_iter_df[['mesh_name', 'cell_id', 'geometry']].head())

In [ ]:
# Example: Get mesh maximum water surface elevation
max_ws_df = HdfResultsMesh.get_mesh_max_ws(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Maximum Water Surface Elevation:")
print(max_ws_df.attrs)
display(max_ws_df.head())

max_ws_df:

| mesh_name | cell_id | maximum_water_surface | maximum_water_surface_time | geometry |
|-----------|---------|---------------------|--------------------------|-----------|
| BaldEagleCr | 0 | 704.054443 | 2018-09-10 18:00:00 | POINT (2083000 370750) |
| BaldEagleCr | 1 | 692.377991 | 2018-09-10 18:04:00 | POINT (2083250 370750) |

In [ ]:
# Plot the max water surface as a map
import matplotlib.pyplot as plt
from ras_commander.HdfResultsMesh import HdfResultsMesh

# Extract x and y coordinates from the geometry column
max_ws_df['x'] = max_ws_df['geometry'].apply(lambda geom: geom.x if geom is not None else None)
max_ws_df['y'] = max_ws_df['geometry'].apply(lambda geom: geom.y if geom is not None else None)

# Remove rows with None coordinates
max_ws_df = max_ws_df.dropna(subset=['x', 'y'])

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_ws_df['x'], max_ws_df['y'], 
                     c=max_ws_df['maximum_water_surface'], 
                     cmap='viridis', 
                     s=10)

# Customize the plot
ax.set_title('Max Water Surface per Cell')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Water Surface (ft)')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the dataframe:")
display(max_ws_df.head())

In [ ]:
# Plot the time of the max water surface elevation (WSEL)

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Convert the 'maximum_water_surface_time' to datetime objects
max_ws_df['max_wsel_time'] = pd.to_datetime(max_ws_df['maximum_water_surface_time'])

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Convert datetime to hours since the start for colormap
min_time = max_ws_df['max_wsel_time'].min()
color_values = (max_ws_df['max_wsel_time'] - min_time).dt.total_seconds() / 3600  # Convert to hours

scatter = ax.scatter(max_ws_df['x'], max_ws_df['y'], 
                     c=color_values, 
                     cmap='viridis', 
                     s=10)

# Customize the plot
ax.set_title('Time of Maximum Water Surface Elevation per Cell')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')

# Set up the colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Hours since simulation start')

# Format the colorbar ticks to show hours
cbar.set_ticks(range(0, int(color_values.max()) + 1, 6))  # Set ticks every 6 hours
cbar.set_ticklabels([f'{h}h' for h in range(0, int(color_values.max()) + 1, 6)])

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Find the overall maximum WSEL and its time
max_wsel_row = max_ws_df.loc[max_ws_df['maximum_water_surface'].idxmax()]
hours_since_start = (max_wsel_row['max_wsel_time'] - min_time).total_seconds() / 3600
print(f"\nOverall Maximum WSEL: {max_wsel_row['maximum_water_surface']:.2f} ft")
print(f"Time of Overall Maximum WSEL: {max_wsel_row['max_wsel_time']}")
print(f"Hours since simulation start: {hours_since_start:.2f} hours")
print(f"Location of Overall Maximum WSEL: X={max_wsel_row['x']}, Y={max_wsel_row['y']}")


In [ ]:
# Example: Get mesh minimum water surface elevation
min_ws_df = HdfResultsMesh.get_mesh_min_ws(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Minimum Water Surface Elevation:")
display(min_ws_df.head())

In [ ]:
# Example: Get mesh maximum face velocity
try:
    max_face_v_df = HdfResultsMesh.get_mesh_max_face_v(plan_hdf_path, ras_object=bald_eagle)
    print("\nMesh Max Face Velocity:")
    display(max_face_v_df.head())
except AttributeError as e:
    print(f"Error: {e}. Please ensure that the method exists in the HdfResultsMesh class.")

In [ ]:
# Extract midpoint coordinates from the LineString geometries
max_face_v_df['x'] = max_face_v_df['geometry'].apply(lambda geom: geom.centroid.x)
max_face_v_df['y'] = max_face_v_df['geometry'].apply(lambda geom: geom.centroid.y)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_face_v_df['x'], max_face_v_df['y'], 
                    c=max_face_v_df['maximum_face_velocity'].abs(),
                    cmap='viridis',
                    s=10)

# Customize the plot
ax.set_title('Max Face Velocity per Face')
ax.set_xlabel('X Coordinate') 
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Face Velocity (ft/s)')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the face velocity dataframe:")
display(max_face_v_df.head())

In [ ]:
# Example: Get mesh minimum face velocity
try:
    min_face_v_df = HdfResultsMesh.get_mesh_min_face_v(plan_hdf_path, ras_object=bald_eagle)
    print("\nMesh Min Face Velocity:")
    display(min_face_v_df.head())
except AttributeError as e:
    print(f"Error: {e}. Please ensure that the method exists in the HdfResultsMesh class.")

In [ ]:
# Example: Get mesh max water surface error
try:
    max_ws_err_df = HdfResultsMesh.get_mesh_max_ws_err(plan_hdf_path, ras_object=bald_eagle)
    print("\nMesh Max Water Surface Error:")
    display(max_ws_err_df.head())
except AttributeError as e:
    print(f"Error: {e}. Please ensure that the method exists in the HdfResultsMesh class.")
    logger.error(f"Failed to get mesh max water surface error: {str(e)}")
except Exception as e:
    print(f"Error: {str(e)}")
    logger.error(f"Failed to get mesh max water surface error: {str(e)}")

In [ ]:
# Plot max water surface error
import matplotlib.pyplot as plt

# Extract x and y coordinates from the geometry points, handling None values
max_ws_err_df['x'] = max_ws_err_df['geometry'].apply(lambda geom: geom.x if geom is not None else None)
max_ws_err_df['y'] = max_ws_err_df['geometry'].apply(lambda geom: geom.y if geom is not None else None)

# Remove any rows with None coordinates
max_ws_err_df = max_ws_err_df.dropna(subset=['x', 'y'])

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_ws_err_df['x'], max_ws_err_df['y'],
                    c=max_ws_err_df['cell_maximum_water_surface_error'],
                    cmap='viridis',
                    s=10)

# Customize the plot
ax.set_title('Max Water Surface Error per Cell')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Water Surface Error (ft)')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the water surface error dataframe:")
display(max_ws_err_df.head())

### Need to add this to the ras-commander library

In [ ]:
# Example: Get mesh summary output for other Datasets (here we retrieve Maximum Face Courant)
try:
    max_courant_df = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Courant", ras_object=bald_eagle)
    print("\nMesh Summary Output (Maximum Courant):")
    print(max_courant_df.attrs)
    display(max_courant_df.head())
except Exception as e:
    print(f"Error: {str(e)}")
    logger.error(f"Failed to get mesh summary output: {str(e)}")
    # Additional error handling or logging can be added here

In [ ]:
# Plot max Courant number
import matplotlib.pyplot as plt
from ras_commander.HdfMesh import HdfMesh
from ras_commander.HdfResultsMesh import HdfResultsMesh
from shapely.geometry import LineString
import geopandas as gpd

# Get mesh max Courant number
max_courant_df = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Courant", ras_object=bald_eagle)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(max_courant_df)

# Get centroids of line geometries for plotting
gdf['centroid'] = gdf.geometry.centroid
gdf['x'] = gdf.centroid.x
gdf['y'] = gdf.centroid.y

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(gdf['x'], gdf['y'],
                    c=gdf['maximum_face_courant'],
                    cmap='viridis',
                    s=10)

# Customize the plot
ax.set_title('Max Courant Number per Face')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Courant Number')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the Courant number dataframe:")
display(gdf.head())


In [ ]:
# Example: Get mesh summary output for other Datasets (here we retrieve Maximum Face Courant)

max_face_shear_df = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Shear Stress", ras_object=bald_eagle)
print("\nMesh Summary Output (Maximum Face Shear Stress:")
print(max_face_shear_df.attrs)
display(max_face_shear_df.head())


In [ ]:
# Plot max face shear stress
import matplotlib.pyplot as plt
from ras_commander.HdfMesh import HdfMesh
from ras_commander.HdfResultsMesh import HdfResultsMesh
from shapely.geometry import Point, LineString
import geopandas as gpd

# Get mesh max face shear stress
max_shear_df = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Shear Stress", ras_object=bald_eagle)

# Calculate centroids of the line geometries and extract coordinates
max_shear_df['centroid'] = max_shear_df['geometry'].apply(lambda line: line.centroid)
max_shear_df['x'] = max_shear_df['centroid'].apply(lambda point: point.x)
max_shear_df['y'] = max_shear_df['centroid'].apply(lambda point: point.y)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_shear_df['x'], max_shear_df['y'],
                    c=max_shear_df['maximum_face_shear_stress'],
                    cmap='viridis',
                    s=10)

# Customize the plot
ax.set_title('Max Face Shear Stress per Face')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Face Shear Stress (PSF)')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the shear stress dataframe:")
display(max_shear_df.head())

In [ ]:
# Example: Get mesh summary output for Minimum Water Surface
summary_df_min_ws = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Minimum Water Surface", ras_object=bald_eagle)
print("\nMesh Summary Output (Minimum Water Surface):")
display(summary_df_min_ws.head())

# Example: Get mesh summary output for Minimum Face Velocity
summary_df_min_fv = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Minimum Face Velocity", ras_object=bald_eagle)
print("\nMesh Summary Output (Minimum Face Velocity):")
display(summary_df_min_fv.head())

# Example: Get mesh summary output for Cell Cumulative Iteration
summary_df_cum_iter = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Cell Cumulative Iteration", ras_object=bald_eagle)
print("\nMesh Summary Output (Cell Cumulative Iteration):")
display(summary_df_cum_iter.head())


In [ ]:
# Get mesh timeseries output

# Get mesh areas from previous code cell
mesh_areas = HdfMesh.get_mesh_area_names(geom_hdf_path, ras_object=bald_eagle)

if mesh_areas:
    mesh_name = mesh_areas[0]  # Use the first 2D flow area name
    timeseries_da = HdfResultsMesh.get_mesh_timeseries(plan_hdf_path, mesh_name, "Water Surface", ras_object=bald_eagle)
    print(f"\nMesh Timeseries Output (Water Surface) for {mesh_name}:")
    print(timeseries_da)
else:
    print("No mesh areas found in the geometry file.")

In [59]:
# Time Series Output Variables for Cells
# 
# Variable Name: Description
# Water Surface: Water surface elevation
# Depth: Water depth
# Velocity: Magnitude of velocity
# Velocity X: X-component of velocity
# Velocity Y: Y-component of velocity
# Froude Number: Froude number
# Courant Number: Courant number
# Shear Stress: Shear stress on the bed
# Bed Elevation: Elevation of the bed
# Precipitation Rate: Rate of precipitation
# Infiltration Rate: Rate of infiltration
# Evaporation Rate: Rate of evaporation
# Percolation Rate: Rate of percolation
# Groundwater Elevation: Elevation of groundwater
# Groundwater Depth: Depth to groundwater
# Groundwater Flow: Groundwater flow rate
# Groundwater Velocity: Magnitude of groundwater velocity
# Groundwater Velocity X: X-component of groundwater velocity
# Groundwater Velocity Y: Y-component of groundwater velocity
# 
# These variables are available for time series output at the cell level in 2D flow areas.


In [ ]:
# Get mesh cells timeseries output
cells_timeseries_ds = HdfResultsMesh.get_mesh_cells_timeseries(plan_hdf_path, mesh_name, ras_object=bald_eagle)
print("\nMesh Cells Timeseries Output:")
print(cells_timeseries_ds)


In [ ]:
# Plot Cell Time Series Data (Random Cell ID)
import matplotlib.pyplot as plt
import numpy as np
import random

# Extract Water Surface data
water_surface = cells_timeseries_ds['BaldEagleCr']['Water Surface']

# Get the time values
time_values = water_surface.coords['time'].values

# Pick a random cell_id
random_cell_id = random.choice(water_surface.coords['cell_id'].values)

# Extract the water surface elevation time series for the random cell
wsel_timeseries = water_surface.sel(cell_id=random_cell_id)

# Find the peak value and its index
peak_value = wsel_timeseries.max().item()
peak_index = wsel_timeseries.argmax().item()

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(time_values, wsel_timeseries, label=f'Cell ID: {random_cell_id}')
plt.scatter(time_values[peak_index], peak_value, color='red', s=100, zorder=5)
plt.annotate(f'Peak: {peak_value:.2f} ft', 
             (time_values[peak_index], peak_value),
             xytext=(10, 10), textcoords='offset points',
             ha='left', va='bottom',
             bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

plt.title(f'Water Surface Elevation Time Series for Random Cell (ID: {random_cell_id})')
plt.xlabel('Time')
plt.ylabel('Water Surface Elevation (ft)')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Log the plotting action
logging.info(f"Plotted water surface elevation time series for random cell ID: {random_cell_id}")

# Display the plot
plt.show()

# Print some statistics
print(f"Statistics for Cell ID {random_cell_id}:")
print(f"Minimum WSEL: {wsel_timeseries.min().item():.2f} ft")
print(f"Maximum WSEL: {peak_value:.2f} ft")
print(f"Mean WSEL: {wsel_timeseries.mean().item():.2f} ft")
print(f"Time of peak: {time_values[peak_index]}")

In [ ]:
# Get mesh faces timeseries output
faces_timeseries_ds = HdfResultsMesh.get_mesh_faces_timeseries(plan_hdf_path, mesh_name, ras_object=bald_eagle)
print("\nMesh Faces Timeseries Output:")
print(faces_timeseries_ds)


In [ ]:
# Plot Random Face Results and Label Peak, Plus Map View

# Step 1: Import necessary libraries 
# In notebook cell at top of notebook

# Step 2: Select a random valid face ID number
random_face = np.random.randint(0, faces_timeseries_ds.sizes['face_id'])

# Step 3: Extract time series data for the selected face
variable = 'face_velocity'  # We could also use 'face_flow'
face_data = faces_timeseries_ds[variable].sel(face_id=random_face)

# Step 4: Find peak value and its corresponding time
peak_value = face_data.max().item()
peak_time = face_data.idxmax().values

# Plot time series
plt.figure(figsize=(12, 8))
plt.plot(faces_timeseries_ds.time, face_data)
plt.title(f'{variable.capitalize()} Time Series for Face {random_face}')
plt.xlabel('Time')
plt.ylabel(f'{variable.capitalize()} ({faces_timeseries_ds.attrs["units"]})')
plt.grid(True)

# Annotate the peak point
plt.annotate(f'Peak: ({peak_time}, {peak_value:.2f})', 
            (peak_time, peak_value),
            xytext=(10, 10), textcoords='offset points',
            arrowprops=dict(arrowstyle="->"))

# Check for negative values and label the minimum if present
min_value = face_data.min().item()
if min_value < 0:
    min_time = face_data.idxmin().values
    plt.annotate(f'Min: ({min_time}, {min_value:.2f})', 
                (min_time, min_value),
                xytext=(10, -10), textcoords='offset points',
                arrowprops=dict(arrowstyle="->"))

plt.tight_layout()
plt.show()

# Create map view plot
fig, ax = plt.subplots(figsize=(12, 8))

# Get mesh faces for map view
mesh_faces = HdfMesh.get_mesh_cell_faces(plan_hdf_path, ras_object=bald_eagle)

# Calculate mesh faces extents with 10% buffer
faces_bounds = mesh_faces.total_bounds
x_min, y_min, x_max, y_max = faces_bounds
buffer_x = (x_max - x_min) * 0.1
buffer_y = (y_max - y_min) * 0.1
plot_xlim = [x_min - buffer_x, x_max + buffer_x]
plot_ylim = [y_min - buffer_y, y_max + buffer_y]

# Set plot limits before adding terrain
ax.set_xlim(plot_xlim)
ax.set_ylim(plot_ylim)

# Add the terrain TIFF to the map, clipped to our desired extent
tiff_path = Path.cwd() / 'example_projects' / 'BaldEagleCrkMulti2D' / 'Terrain' / 'Terrain50.baldeagledem.tif'
with rasterio.open(tiff_path) as src:
    show(src, ax=ax, cmap='terrain', alpha=0.5)
    
# Reset the limits after terrain plot
ax.set_xlim(plot_xlim)
ax.set_ylim(plot_ylim)

# Plot all faces in gray
mesh_faces.plot(ax=ax, color='lightgray', alpha=0.5, zorder=2)

# Get the selected face geometry
selected_face = mesh_faces[mesh_faces['face_id'] == random_face]

# Highlight the selected face in red
selected_face.plot(
    ax=ax, 
    color='red',
    linewidth=2,
    label=f'Selected Face (ID: {random_face})',
    zorder=3
)

# Get bounds of selected face for zoomed inset
bounds = selected_face.geometry.bounds.iloc[0]
x_center = (bounds.iloc[0] + bounds.iloc[2]) / 2
y_center = (bounds.iloc[1] + bounds.iloc[3]) / 2
buffer = max(bounds.iloc[2] - bounds.iloc[0], bounds.iloc[3] - bounds.iloc[1]) * 2

# Create zoomed inset with a larger size, inside the map frame
axins = inset_axes(ax, width="70%", height="70%", loc='lower right',
                  bbox_to_anchor=(0.65, 0.05, 0.35, 0.35),
                  bbox_transform=ax.transAxes)

# Plot terrain and faces in inset
with rasterio.open(tiff_path) as src:
    show(src, ax=axins, cmap='terrain', alpha=0.5)
    
# Plot zoomed view in inset
mesh_faces.plot(ax=axins, color='lightgray', alpha=0.5, zorder=2)
selected_face.plot(ax=axins, color='red', linewidth=2, zorder=3)

# Set inset limits with slightly more context
axins.set_xlim(x_center - buffer/1.5, x_center + buffer/1.5)
axins.set_ylim(y_center - buffer/1.5, y_center + buffer/1.5)

# Remove inset ticks for cleaner look
axins.set_xticks([])
axins.set_yticks([])

# Add a border to the inset
for spine in axins.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1.5)

# Create connection lines between main plot and inset
# Get the selected face centroid for connection point
centroid = selected_face.geometry.centroid.iloc[0]
con1 = ConnectionPatch(
    xyA=(centroid.x, centroid.y), coordsA=ax.transData,
    xyB=(0.02, 0.98), coordsB=axins.transAxes,
    arrowstyle="-", linestyle="--", color="gray", alpha=0.6
)
con2 = ConnectionPatch(
    xyA=(centroid.x, centroid.y), coordsA=ax.transData,
    xyB=(0.98, 0.02), coordsB=axins.transAxes,
    arrowstyle="-", linestyle="--", color="gray", alpha=0.6
)

ax.add_artist(con1)
ax.add_artist(con2)

# Add title and legend to main plot
ax.set_title('Mesh Face Map View with Terrain')
ax.legend()

# Ensure equal aspect ratio while maintaining our desired extents
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

# Print summary information
print(f"Random Face: {random_face}")
print(f"Peak Value: {peak_value:.2f} {faces_timeseries_ds.attrs['units']} at {peak_time}")
if min_value < 0:
    print(f"Minimum Value: {min_value:.2f} {faces_timeseries_ds.attrs['units']} at {min_time}")

# Log the plotting action
logging.info(f"Plotted mesh face time series and map view for random face ID: {random_face} with terrain")

In [ ]:
# Get meteorology precipitation attributes
meteo_precip_attrs = HdfPlan.get_plan_met_precip(plan_hdf_path, ras_object=bald_eagle)
print("\nMeteorology Precipitation Attributes:")
for key, value in meteo_precip_attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Get results unsteady attributes
results_unsteady_attrs = HdfResultsPlan.get_unsteady_info(plan_hdf_path, ras_object=bald_eagle)
print("\nResults Unsteady Attributes:")
for key, value in results_unsteady_attrs.items():
    print(f"{key}: {value}")

In [ ]:
# Get results unsteady summary attributes
results_unsteady_summary_attrs = HdfResultsPlan.get_unsteady_summary(plan_hdf_path, ras_object=bald_eagle)
print("\nResults Unsteady Summary Attributes:")
for key, value in results_unsteady_summary_attrs.items():
    print(f"{key}: {value}")

# Get results volume accounting attributes
volume_accounting_attrs = HdfResultsPlan.get_volume_accounting(plan_hdf_path, ras_object=bald_eagle)
print("\nVolume Accounting Attributes:")
for key, value in volume_accounting_attrs.items():
    print(f"{key}: {value}")